# Python Projet 5 - Prédiction des tags 

## Multilabel classification avec la méthode One-vs-Rest

Dans le cadre du cours OpenClassrooms Machine Learning et du projet d'évaluation #5, ce notebook présente les différents modèles de prédiction des tags ainsi qu'une comparaison avec l'analyse non supervisée obtenue par LDA. 

Author : Vincent Arrigoni, 04/2023 

Regexp : https://www.debuggex.com/cheatsheet/regex/python

Tout savoir :https://datascientest.com/regex-tout-savoir

Test regexp : https://regex101.com/

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# import of libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MultiLabelBinarizer 
from sklearn.multioutput import MultiOutputClassifier 
from sklearn.metrics import precision_score, recall_score, jaccard_score, classification_report
from gensim.models import Word2Vec
import re
from tqdm import tqdm
from time import time
from sklearn.utils import shuffle

# import tensorflow as tf
# import tensorflow_hub as hub
# import tensorflow_text as text
# from official.nlp import optimization  # to create AdamW optimizer

In [3]:
data = pd.read_csv('/content/drive/MyDrive/OCS/Projet5/msg_preprocessed.csv')
data.head(10)

,Unnamed: 0,Text,Tags,Token,Tags_lst,Tags_lst_new,Token_lst_new
0,0,how do I calculate a rolling idxmax<p>consider...,<python><pandas><numpy><dataframe><series>,"['calcul', 'roll', 'idxmax', 'consid', 'pd', '...","['python', 'pandas', 'numpy', 'dataframe', 'se...",['python'],"['calcul', 'roll', 'idxmax', 'consid', 'pd', '..."
1,1,Object-Oriented Callbacks for C++?<p>Is there ...,<c++><oop><callback><pointer-to-member><eiffel>,"['object', 'orient', 'callback', 'c++', 'libra...","['c++', 'oop', 'callback', 'pointer-to-member'...",['c++'],"['object', 'orient', 'callback', 'c++', 'libra..."
2,2,TDD and ADO.NET Entity Framework<p>I've been p...,<unit-testing><entity-framework><ado.net><tdd>...,"['tdd', 'ado', '.net', 'entiti', 'framework', ...","['unit-testing', 'entity-framework', 'ado.net'...",['unit-testing'],"['tdd', 'ado', '.net', 'entiti', 'framework', ..."
3,3,Better techniques for trimming leading zeros i...,<sql><sql-server><sql-server-2005><tsql><string>,"['better', 'techniqu', 'trim', 'lead', 'zero',...","['sql', 'sql-server', 'sql-server-2005', 'tsql...","['sql', 'sql-server', 'string']","['better', 'techniqu', 'trim', 'lead', 'zero',..."
4,4,"""No X11 DISPLAY variable"" - what does it mean?...",<java><linux><variables><x11><headless>,"['x11', 'display', 'variabl', 'mean', 'tri', '...","['java', 'linux', 'variables', 'x11', 'headless']","['java', 'linux']","['x11', 'display', 'variabl', 'mean', 'tri', '..."
5,5,Why doesn't .NET find the OpenSSL.NET dll?<p><...,<c#><.net><dll><dllimport><dllnotfoundexception>,"['.net', 'find', 'openssl', '.net', 'dll', 'ed...","['c#', '.net', 'dll', 'dllimport', 'dllnotfoun...","['c#', '.net']","['.net', 'find', 'openssl', '.net', 'dll', 'ed..."
6,6,Iterating through/Parsing JSON Object via Java...,<javascript><json><jquery><parsing><loops>,"['iter', 'pars', 'json', 'object', 'via', 'jav...","['javascript', 'json', 'jquery', 'parsing', 'l...","['javascript', 'json', 'jquery']","['iter', 'pars', 'json', 'object', 'via', 'jav..."
7,7,Class inherited from class without default con...,<c++><class><inheritance><constructor><default...,"['class', 'inherit', 'class', 'without', 'defa...","['c++', 'class', 'inheritance', 'constructor',...",['c++'],"['class', 'inherit', 'class', 'without', 'defa..."
8,8,libxml2 vs expat for an XMPP server<p>I'm tryi...,<c><go><xmpp><libxml2><expat-parser>,"['libxml2', 'vs', 'expat', 'xmpp', 'server', '...","['c', 'go', 'xmpp', 'libxml2', 'expat-parser']",['c'],"['libxml2', 'vs', 'expat', 'xmpp', 'server', '..."
9,9,Implications of deploying a Debug build of an ...,<c#><security><debugging><deployment><compiler...,"['implic', 'deploy', 'debug', 'build', 'applic...","['c#', 'security', 'debugging', 'deployment', ...","['c#', 'security', 'debugging']","['implic', 'deploy', 'debug', 'build', 'applic..."


In [4]:
def regexp_Token(x):
  return(re.findall(r'(?:C\+\+)|(?:c\+\+)|(?:c\#)|(?:C\#)|(?:\.net)|(?:\.NET)|\w{2,}', x['Token_lst_new']))
         
def regexp_Tags(x):
  return(re.findall(r'(?:C\+\+)|(?:c\+\+)|(?:c\#)|(?:C\#)|(?:\.net)|(?:\.NET)|\w{2,}', x['Tags_lst_new']))

ds = pd.DataFrame({'Token' : data[:5000].apply(regexp_Token, axis=1), 'Tags' : data[:5000].apply(regexp_Tags, axis=1)})
ds = shuffle(ds)
ds.reset_index(inplace=True)
ds.head(10)

,index,Token,Tags
0,4925,"[load, data, local, skip, first, line, tri, lo...","[sql, mysql]"
1,4222,"[spark, group, datafram, syntax, syntax, use, ...","[java, sql]"
2,3884,"[load, blob, imag, data, qpixmap, write, progr...","[python, image]"
3,369,"[visual, studio, support, new, c++, standard, ...","[c++, visual, studio]"
4,4868,"[ilmerg, local, resourc, assembl, applic, whos...",[]
5,116,"[twig, wordpress, pass, array, argument, funct...","[php, arrays]"
6,4095,"[c#, get, complet, desktop, size, find, size, ...","[c#, .net]"
7,1340,"[framework, window, form, db, driven, applic, ...","[c#, database, winforms]"
8,3329,"[adapt, layout, xcode, develop, app, use, xcod...","[ios, xcode]"
9,32,"[differ, includ, includ, filenam, differ, use,...",[c++]


## Approche Bag Of Words pour la classification par Régression Logistique 

In [5]:
def dummy(doc):
  return doc
  # https://stackoverflow.com/questions/35867484/pass-tokens-to-countvectorizer
  # pas besoin que countvectorizer fasse un preprocessing ou une tokenization des documents

vectorizer = CountVectorizer(tokenizer=dummy, preprocessor=dummy, binary=True)
X = vectorizer.fit_transform(ds['Token'])
X = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

LabelBinarizer = MultiLabelBinarizer()
y = LabelBinarizer.fit_transform(ds['Tags'])
y = pd.DataFrame(y, columns=LabelBinarizer.classes_)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [6]:
# Train-Test split 
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Build the model
model = MultiOutputClassifier(LogisticRegression())
# https://scikit-learn.org/stable/modules/multiclass.html

In [7]:
# search = GridSearchCV(
#     estimator = model,
#     param_grid={'estimator__C':np.logspace(-3, 0, 4)},
#     scoring = ['accuracy', 'precision_micro', 'recall_micro'],  
#     refit = 'precision_micro',
#     cv = 5
# ).fit(X_train, y_train)
# # https://datascience.stackexchange.com/questions/107867/how-to-train-multioutput-classification-with-hyperparameter-tuning-in-sklearn
# # https://stackoverflow.com/questions/41899132/invalid-parameter-for-sklearn-estimator-pipeline
# # https://scikit-learn.org/stable/modules/model_evaluation.html#from-binary-to-multiclass-and-multilabel

# # error : Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

# print(search.best_estimator_)
# print(search.best_params_)
# clf_BOW = search.best_estimator_ 

In [8]:
model.fit(X_train, y_train)
clf_BOW = model

In [9]:
y_pred = clf_BOW.predict(X_test)
print("Classifier report:\n", classification_report(y_test, y_pred, target_names=LabelBinarizer.classes_, zero_division=0))
# Calculate the Jaccard score
score = jaccard_score(y_test, y_pred, average='micro')
print(f"Jaccard score: {score:.4f}")

Classifier report:
                 precision    recall  f1-score   support

          .net       0.64      0.42      0.51       166
          ajax       0.00      0.00      0.00        20
     algorithm       0.71      0.29      0.42        17
       android       0.92      0.72      0.81        65
        arrays       0.50      0.05      0.08        22
           asp       0.73      0.54      0.62        56
          bash       0.33      0.07      0.12        14
            c#       0.82      0.61      0.70       194
           c++       0.80      0.67      0.73       117
         cocoa       0.00      0.00      0.00        12
           css       0.75      0.29      0.42        31
      database       0.71      0.22      0.33        23
     debugging       0.75      0.19      0.30        16
        django       0.90      0.53      0.67        17
           gcc       0.25      0.08      0.12        13
     hibernate       0.75      0.46      0.57        13
          html       0.57  

## Approche TF-IDF pour la classification par Régression Logistique

In [10]:
def dummy(doc):
  return doc

# Binarized before Train/Test split
LabelBinarizer = MultiLabelBinarizer()
y_tfidf = LabelBinarizer.fit_transform(ds['Tags'])
y_tfidf = pd.DataFrame(y_tfidf, columns=LabelBinarizer.classes_)

# Train-Test split 
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(ds['Token'], y_tfidf)

# Binarized after Train/Test split as the frequency depends on the sample size
# Perform tf-idf on all dataset could induce data leak
tfidf_vectorizer = TfidfVectorizer(preprocessor=dummy, tokenizer=dummy)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_tfidf)
X_test_tfidf = tfidf_vectorizer.transform(X_test_tfidf)
# https://www.kaggle.com/code/neerajmohan/nlp-text-classification-using-tf-idf-features

# Build the model
model = MultiOutputClassifier(LogisticRegression())

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [11]:
# search_tfidf = GridSearchCV(
#     estimator = model,
#     param_grid={'estimator__C':np.logspace(-3, 0, 4)},
#     scoring = ['accuracy', 'precision_micro', 'recall_micro'],
#     refit = 'precision_micro',
#     cv = 5
# ).fit(X_train_tfidf, y_train_tfidf)

# print(search_tfidf.best_estimator_)
# print(search_tfidf.best_params_)
# clf_TFIDF = search_tfidf.best_estimator_

In [12]:
model.fit(X_train_tfidf, y_train_tfidf)
clf_TFIDF = model

In [13]:
y_pred_tfidf = clf_TFIDF.predict(X_test_tfidf)
print("Classifier report:\n", classification_report(y_test_tfidf, y_pred_tfidf, target_names=LabelBinarizer.classes_, zero_division=0))
# Calculate the Jaccard score
score = jaccard_score(y_test_tfidf, y_pred_tfidf, average='micro')
print(f"Jaccard score: {score:.4f}")

Classifier report:
                 precision    recall  f1-score   support

          .net       0.81      0.21      0.33       187
          ajax       0.00      0.00      0.00        18
     algorithm       0.00      0.00      0.00        21
       android       0.93      0.27      0.42        51
        arrays       1.00      0.08      0.15        24
           asp       0.82      0.15      0.25        60
          bash       0.00      0.00      0.00        17
            c#       0.83      0.22      0.35       221
           c++       0.95      0.33      0.49       109
         cocoa       0.00      0.00      0.00        20
           css       0.00      0.00      0.00        26
      database       0.00      0.00      0.00        25
     debugging       0.00      0.00      0.00        18
        django       1.00      0.04      0.08        25
           gcc       0.00      0.00      0.00        22
     hibernate       0.00      0.00      0.00        12
          html       0.00  

## Word Embeddings with Word2Vec and Logistic Regression

In [14]:
# Word2Vec

# Binarized before Train/Test split
LabelBinarizer = MultiLabelBinarizer()
y_wrd2vec = LabelBinarizer.fit_transform(ds['Tags'])
# y_wrd2vec = pd.DataFrame(y_wrd2vec, columns=LabelBinarizer.classes_)

# Train-Test split 
X_train_wrd2vec, X_test_wrd2vec, y_train_wrd2vec, y_test_wrd2vec = train_test_split(ds['Token'], y_wrd2vec)

# Train the word2vec model 
w2v_model = Word2Vec(X_train_wrd2vec, 
                     vector_size=100,
                     window=5, 
                     min_count=2)

In [15]:
len(w2v_model.wv.index_to_key) # all the words that appear at least twice in the document

17704

In [16]:
w2v_model.wv[1] # embedding of the second word of the first sentence

array([-0.23578651, -0.10913433,  0.7658911 , -0.18417378,  1.1890931 ,
       -1.6385256 ,  0.7420702 ,  1.9548098 , -0.39090303, -0.6665687 ,
       -0.9120041 , -1.9424314 , -0.824719  ,  0.8043178 , -0.1640544 ,
       -0.09647217,  0.00349097, -0.08169197, -0.06887006, -1.3868057 ,
        0.5891635 ,  1.2561325 ,  0.6658882 , -0.13917188,  0.5761383 ,
        0.34051576,  0.02874151, -0.5445694 , -0.40601948,  0.24535586,
        0.7371171 ,  0.3028645 , -0.3874159 , -0.9124782 , -0.5838178 ,
        0.4150036 ,  0.0933493 , -1.3769357 , -0.3361844 , -1.1060729 ,
        1.2001649 , -0.8600497 , -1.1259574 , -0.30915004,  0.1966482 ,
       -0.00282346, -0.12582842, -1.0580394 ,  0.3765773 ,  0.56927556,
       -0.09306105, -1.2759497 , -0.19781052,  1.1576179 , -0.5416748 ,
        0.5268161 , -0.30585557,  0.09392111, -0.7436096 ,  0.4114962 ,
        0.4513177 ,  0.3198033 ,  0.16093701, -0.3991346 , -0.1819438 ,
        0.97867906,  0.10426394, -0.35354275, -0.8624781 ,  0.75

In [17]:
w2v_model.wv.most_similar(positive=["python"]) 

[('nuget', 0.9202696681022644),
 ('collada', 0.9146614074707031),
 ('requirej', 0.9067397117614746),
 ('tkinter', 0.9016359448432922),
 ('pip', 0.8968111276626587),
 ('berkeley', 0.8920202255249023),
 ('environ', 0.8907024264335632),
 ('instal', 0.8906975984573364),
 ('jsdoc', 0.8880780935287476),
 ('standalon', 0.883903443813324)]

In [18]:
words = set(w2v_model.wv.index_to_key) # vocabulary correspondance
# https://medium.com/@dilip.voleti/classification-using-word2vec-b1d79d375381

X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words]) for ls in X_train_wrd2vec]) # concatenate word embeddings of the same sentence in the same element of X_train_vect

X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])  for ls in X_test_wrd2vec])

<ipython-input-18-87f37664ccdc>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words]) for ls in X_train_wrd2vec]) # concatenate word embeddings of the same sentence in the same element of X_train_vect
<ipython-input-18-87f37664ccdc>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])  for ls in X_test_wrd2vec])


In [19]:
# Compute sentence embeddings by averaging the word embeddings for the words contained in the sentence
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))
        
X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

In [20]:
# Build the model
model_WRD2VEC = MultiOutputClassifier(LogisticRegression())
model_WRD2VEC.fit(X_train_vect_avg, y_train_wrd2vec)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

MultiOutputClassifier(estimator=LogisticRegression())

In [21]:
# # Instantiate and fit a basic Random Forest model on top of the vectors
# from sklearn.ensemble import RandomForestClassifier
# rf = MultiOutputClassifier(RandomForestClassifier())
# rf_model = rf.fit(X_train_vect_avg, y_train_wrd2vec)

In [22]:
# # Use the trained model to make predictions on the test data
# y_pred = rf_model.predict(X_test_vect_avg)

In [23]:
y_pred_wrd2vec = model_WRD2VEC.predict(X_test_vect_avg)
print("Classifier report:\n", classification_report(y_test_wrd2vec, y_pred_wrd2vec, target_names=LabelBinarizer.classes_, zero_division=0))
# Calculate the Jaccard score
score = jaccard_score(y_test_wrd2vec, y_pred_wrd2vec, average='samples')
print(f"Jaccard score: {score:.4f}")

Classifier report:
                 precision    recall  f1-score   support

          .net       0.65      0.25      0.36       167
          ajax       0.00      0.00      0.00        10
     algorithm       0.50      0.06      0.11        17
       android       0.84      0.46      0.59        59
        arrays       0.25      0.04      0.07        24
           asp       0.44      0.15      0.22        48
          bash       0.00      0.00      0.00        15
            c#       0.60      0.22      0.32       195
           c++       0.81      0.35      0.49       101
         cocoa       1.00      0.08      0.15        12
           css       0.17      0.07      0.10        28
      database       0.00      0.00      0.00        20
     debugging       0.00      0.00      0.00        14
        django       0.60      0.12      0.20        25
           gcc       0.00      0.00      0.00        11
     hibernate       0.00      0.00      0.00         9
          html       0.39  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in samples with no true or predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## BERT NLP algorithm

In [24]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.4 MB/s eta 0:00:00


In [25]:
from transformers import BertTokenizer, TFBertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', truncation=True, padding=True)
model_BERT = TFBertModel.from_pretrained("bert-base-uncased")
# text = "R:eplace me by any text you'd like."
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# print(output)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [26]:
# from transformers import AutoModel # For BERTs
# # from transformers import AutoModeForSequenceClassification # For models fine-tuned on MNLI
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-small") # v1 and v2
# model = AutoModel.from_pretrained("prajjwal1/bert-small") # v1 and v2

In [28]:
# ID ENCODING BY HAND 
def bert_encode(token_lst, tokenizer, max_len=64):
  all_tokens = []
  all_masks = []
  all_segments = []

  for text in token_lst:

    text = text[:max_len-2] # How to deal with truncation ? 
    text = ["[CLS]"] + text + ["[SEP]"] # special tokens
    pad_len = max_len - len(text) # necesary PAD length 

    token_id = tokenizer.convert_tokens_to_ids(text) + [0] * pad_len 
    pad_masks = [1] * len(text) + [0] * pad_len
    segment_ids = [0] * max_len
        
    all_tokens.append(token_id)
    all_masks.append(pad_masks)
    all_segments.append(segment_ids)
  
  return {'input_ids' : np.array(all_tokens),
          'token_type_ids' : np.array(all_masks),
          'attention_mask' : np.array(all_segments)}

emb_BERT = np.array([])
batch_size=500
for i in tqdm(range(len(ds)//batch_size)):
  # Batch it because otherwise too big for the model to train on 
  encoded_tokens = bert_encode(ds.loc[i*batch_size:(i+1)*batch_size-1,'Token'], tokenizer)
  # print(encoded_tokens)
  # print(len(encoded_tokens))
  embeddings = model_BERT(encoded_tokens)
  if len(emb_BERT)==0:
    emb_BERT = np.array(embeddings['pooler_output'])
  else:
    emb_BERT = np.vstack((emb_BERT, np.array(embeddings['pooler_output'])))
    # De-batching it to usual format 

In [29]:
print(len(embeddings['pooler_output']))

500


In [30]:
# OVERALL ENCODING
def BERT_encode(batch):
  batch = list(map(lambda x : ' '.join(x) ,batch))
  encoded_input = tokenizer(batch, return_tensors='tf', truncation=True, padding=True)
  # grouped_inputs = {'input_ids':tf.cast(encoded_input.input_ids, tf.int16),
  #                   'token_type_ids' : tf.cast(encoded_input.token_type_ids, tf.int16),
  #                   'attention_mask' : tf.cast(encoded_input.attention_mask, tf.int16)}
  # print(tf.cast(encoded_input.input_ids, tf.int16))
  # # print(encoded_input.astype(int16))
  # print(len(encoded_input))
  return model_BERT(encoded_input)

emb_BERT = np.array([])
# Batch the dataset to compute embeddings more easily 
batch_size = 100
max_len = 64 # max number of word to avoid Out Of Memory issue
nb_batches = len(ds)//batch_size
for i in tqdm(range(len(ds)//batch_size)):
  # Batch it because otherwise too big for the model to train on 
  embeddings = BERT_encode(ds.loc[i*batch_size:(i+1)*batch_size-1,'Token'].str.slice(0,max_len))
  # embeddings = model_USE(encoded_tokens)
  if emb_BERT.size==0:
    emb_BERT = np.array(embeddings['pooler_output'])
    # print(emb_BERT.size)
  else:
    emb_BERT = np.vstack((emb_BERT, np.array(embeddings['pooler_output'])))
    # print('merging')
    # print(i)
    # print(emb_BERT.size)
    # De-batching it to usual format

100%|██████████| 50/50 [31:46<00:00, 38.13s/it]


In [31]:
x = emb_BERT
y = ds['Tags']

# Initialize MultiLabelBinarizer
LabelBinarizer = MultiLabelBinarizer()

y = LabelBinarizer.fit_transform(y)

X_train_BERT, X_test_BERT, y_train_BERT, y_test_BERT = train_test_split(x, y, test_size=0.2, random_state=42)

import warnings
from sklearn.exceptions import ConvergenceWarning

# Suppress specific warnings
warnings.filterwarnings('ignore', category=ConvergenceWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

best_params_SB = {'estimator__C': 0.005, 'estimator__max_iter': 500, 'estimator__penalty': 'none', 'estimator__solver': 'lbfgs'}

# Remove the 'estimator__' prefix from the keys of best_params
best_params_cleaned_SB = {key.replace('estimator__', ''): value for key, value in best_params_SB.items()}

# Train the OneVsRestClassifier with the best parameters on the entire training data
model_BERT = MultiOutputClassifier(LogisticRegression(**best_params_cleaned_SB, random_state=42))

In [32]:
model_BERT.fit(X_train_BERT, y_train_BERT)

MultiOutputClassifier(estimator=LogisticRegression(C=0.005, max_iter=500,
                                                   penalty='none',
                                                   random_state=42))

In [33]:
# Make predictions on the testing data
y_pred_BERT = model_BERT.predict(X_test_BERT)

# Evaluate the classifier using classification_report
print("Classifier report:\n", classification_report(y_test_BERT, y_pred_BERT, target_names=LabelBinarizer.classes_, zero_division=0))

# Calculate the Jaccard score
score = jaccard_score(y_test_BERT, y_pred_BERT, average='samples')
print(f"Jaccard score: {score:.4f}")

# Reset warnings to default
warnings.resetwarnings()

Classifier report:
                 precision    recall  f1-score   support

          .net       0.49      0.42      0.45       123
          ajax       0.31      0.31      0.31        13
     algorithm       0.11      0.06      0.08        17
       android       0.41      0.40      0.40        53
        arrays       0.53      0.60      0.56        15
           asp       0.17      0.23      0.19        40
          bash       0.33      0.23      0.27        13
            c#       0.45      0.38      0.41       143
           c++       0.50      0.43      0.46        83
         cocoa       0.21      0.30      0.25        10
           css       0.36      0.31      0.33        26
      database       0.11      0.14      0.12        22
     debugging       0.08      0.08      0.08        13
        django       0.43      0.40      0.41        15
           gcc       0.18      0.30      0.22        10
     hibernate       0.14      0.25      0.18         8
          html       0.19  

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## USE NLP Algorithm

In [34]:
import tensorflow_hub as hub 

model_USE = hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')

<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()


In [ ]:
def USE_encode(batch):
  batch = list(map(lambda x : ' '.join(x) ,batch))
  return model_USE(batch).numpy()

emb_USE = np.array([])
# Batch the dataset to compute embeddings more easily 
batch_size = 200
nb_batches = len(ds)//batch_size
for i in range(len(ds)//batch_size):
  # Batch it because otherwise too big for the model to train on 
  embeddings = USE_encode(ds.loc[i*batch_size:(i+1)*batch_size-1,'Token'])
  # embeddings = model_USE(encoded_tokens)
  if len(emb_USE)==0:
    emb_USE = np.array(embeddings)
  else:
    emb_USE = np.vstack((emb_USE, np.array(embeddings)))
    # De-batching it to usual format

In [36]:
embeddings.shape

(200, 512)

In [37]:
emb_USE.shape

(5000, 512)

In [52]:
x = emb_USE
y = ds['Tags']

# Initialize MultiLabelBinarizer
LabelBinarizer = MultiLabelBinarizer()

y = LabelBinarizer.fit_transform(y)

X_train_USE, X_test_USE, y_train_USE, y_test_USE = train_test_split(x, y, test_size=0.2, random_state=42)

import warnings
from sklearn.exceptions import ConvergenceWarning

# Suppress specific warnings
warnings.filterwarnings('ignore', category=ConvergenceWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

best_params_SB = {'estimator__C': 0.01, 'estimator__max_iter': 500, 'estimator__penalty': 'none', 'estimator__solver': 'lbfgs'}

# Remove the 'estimator__' prefix from the keys of best_params
best_params_cleaned_SB = {key.replace('estimator__', ''): value for key, value in best_params_SB.items()}

# Train the OneVsRestClassifier with the best parameters on the entire training data
USE_clf = MultiOutputClassifier(LogisticRegression(**best_params_cleaned_SB, random_state=42))

In [53]:
USE_clf.fit(X_train_USE, y_train_USE)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


MultiOutputClassifier(estimator=LogisticRegression(C=0.01, max_iter=500,
                                                   penalty='none',
                                                   random_state=42))

In [54]:
# Make predictions on the testing data
y_pred_USE = USE_clf.predict(X_test_USE)

# Evaluate the classifier using classification_report
print("Classifier report:\n", classification_report(y_test_USE, y_pred_USE, target_names=LabelBinarizer.classes_, zero_division=0))

# Calculate the Jaccard score
score = jaccard_score(y_test_USE, y_pred_USE, average='samples')
print(f"Jaccard score: {score:.4f}")

# Reset warnings to default
warnings.resetwarnings()

Classifier report:
                 precision    recall  f1-score   support

          .net       0.47      0.50      0.48       123
          ajax       0.89      0.62      0.73        13
     algorithm       0.25      0.12      0.16        17
       android       0.76      0.66      0.71        53
        arrays       0.62      0.53      0.57        15
           asp       0.38      0.38      0.38        40
          bash       0.50      0.31      0.38        13
            c#       0.52      0.59      0.55       143
           c++       0.63      0.66      0.65        83
         cocoa       0.43      0.30      0.35        10
           css       0.54      0.54      0.54        26
      database       0.13      0.09      0.11        22
     debugging       0.67      0.46      0.55        13
        django       0.93      0.87      0.90        15
           gcc       0.35      0.60      0.44        10
     hibernate       0.17      0.12      0.14         8
          html       0.55  

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [42]:
import joblib


with open('USE_clf.pkl', 'wb') as f:
  joblib.dump((LabelBinarizer, USE_clf), f)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [43]:
yolo, lolilol = joblib.load('USE_clf.pkl')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
